In [ ]:
''' FOR DEVELOPMENT ONLY, DELETE CELL '''

# set auto reload imported modules tagged
%load_ext autoreload
%autoreload 2

# data array processing
import numpy as np
#import pandas as pd

#from scipy.stats import linregress
#from scipy.interpolate import splev, splrep

#from scipy.signal import savgol_filter

#from scipy.optimize import curve_fit


# plotting with matplotlib
%matplotlib widget
from matplotlib import pyplot as plt

#import matplotlib.colors as colors
#import matplotlib.cm as cmx

import seaborn as sns

## development override params
pvlibs_path = '/home/brendan/dev/pvlibs'
base_path = '../data/iv/halm'


In [ ]:
''' initialise notebook '''

# add location to path
#pvlibs_path = '/srv/pvlibs'
import sys
sys.path.append(pvlibs_path)

# plotting with matplotlib, set interactive figure display
%matplotlib widget

# import pvlibs package
import pvlibs


In [ ]:
''' import measurement files '''

# define directory to search; "./" for current directory, "../" for up one directory
#base_path = './data'

# define measurement properties (!required!)
props = {
    # measurement type
    'meas_type': 'iv', #
    'file_ext': 'prn', # ['prn'|'xlsx']
    'file_type': 'halm-500', # ['halm-500'|'wavelabs']
    
    # wafer properties
    'wafer_area': .243, # [cm^-2]
    
    # measurement conditions
    'temperature': 298.15,
}

# generate measurement file database
db = pvlibs.init_file_db(base_path, props)


In [ ]:
''' parse parameters from file name '''

# define single parameter separator; e.g. '-', '_', or '\s' (space)
param_sep = '-'

# define ordered list of parameters within filename separated by param_sep
params = [
    'wafer',
    'device',
    'side',
    'polarity',
]

# parse file name parameters from each measurement file
db = pvlibs.parse_file_names(db, param_sep, params)


In [ ]:
set([n['device'] for n in db])

In [ ]:
db[0]

In [ ]:

node = db[0]

pvlibs.data_import.core.import_data_file(node['meas_type'], node['file_type'], node['file_path'], node['file_name'])

In [ ]:
''' import measurement data '''

# import data from files
db = pvlibs.import_file_data(db)


In [ ]:

fig = plt.figure()
ax = []; ax.append( fig.add_subplot() )#; ax.append( ax[0].twinx() )

# select nodes by device id
nodes = [ n for n in db if n['device'] == '1' ]
#nodes = db[:5]

# sort by proc time
#nodes = [ nodes[j] for j in np.argsort([nodes[i]['proc_time'] for i in range(len(nodes))]) ]


labels = [
    'full',
    'half',
    'dark',
    #'shunt',
    #'series'
]

#cnorm  = colors.Normalize(vmin = 0, vmax = len(nodes))
#smap = cmx.ScalarMappable(norm = cnorm, cmap = 'magma')

for i in range(len(nodes))[::2]:
    
    d = nodes[i]

    for l in labels:

        V = d[l]['voltage']
        I = -d[l]['current']
        #L = d[l]['intensity']

        j = np.argsort(V)
        V = V[j]
        I = I[j]
        #L = L[j]

        #lbl = '{}'.format(d['proc_time']) if l == labels[0] else None
        
        #ax[0].plot(V, I, '-', c = smap.to_rgba(i), label = lbl)
        ax[0].plot(V, I, '--', linewidth = 2, alpha = .8, label = l)
            
        #ax[1].plot(V, L, '--')#, c = smap.to_rgba(i))
    
ax[0].set_xlabel('Voltage (V)')
ax[0].set_ylabel('Current (A)')
#ax[1].set_ylabel('Illumination Intensity (mW)')

#ax[1].set_ylim(-.1, 1.1)

#ax[0].set_yscale('log')

ax[0].legend()
plt.tight_layout()
plt.show()


In [ ]:
plt.close('all')

In [ ]:
''' process measurement data '''

# process measurement data
db = pvlibs.process_file_data(db)


In [ ]:
''' compile and export data'''

# define list of measurement labels to export; format each as 'file_name_param': 'Output Label'
labels = {
    'wafer': 'Wafer Type',
    'device': 'Device',
    'side': 'Wafer Side'
}

# select values to export; format each as 'param': 'Output Label'
values = {
    'area': 'Area [cm^-2]',
    'isc': 'Isc [A]',
    'voc': 'Voc [V]',
    'pmpp': 'Pmpp [W]',
    'impp': 'Impp [I]',
    'vmpp': 'Vmpp [V]',
    'ff': 'FF []',
    'eta': 'Eta [%]',
    'rs': 'Rs [Ohm]',
    'rp': 'Rp [Ohm]',
    'jsc': 'Jsc [A/cm^-2]',
    'jmpp': 'Jmpp [A/cm^-2]',
    'rs_sqr': 'Rs [Ohm/cm^-2]',
    'rp_sqr': 'Rp [Ohm/cm^-2]',
}

# define results output file name
file_name = './iv-summary'

# compile dataset by selected labels and values
data = pvlibs.compile_data(db, labels, values, file_name)
